# Create a MatrixTable and QC the hell out of it
## 0 Import stuff and set your parameters

In [ ]:
from pathlib import Path, PurePath
import sys

module_path = Path('..').resolve().__str__()

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd

from src.utils import get_position, lookup_vcfs
from src.matrixtables import *

In [ ]:
import pyspark
import dxpy
import dxdata
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

import hail as hl

hl.init(sc=sc, default_reference='GRCh38',log="../hail_logs")

In [ ]:
MAPPING_FILE = "/mnt/project/misc_data/mappings_with_blocks.tsv"
INTERVAL_FILE = "/mnt/project/misc_data/xgen_plus_spikein.b38.bed"
VCF_DIR = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, pVCF format"
TMP_DIR = "../tmp"
VCF_VERSION = "v1"
GENE = "OPRM1"
DOWNSAMPLE_P = 1 / 200

In [ ]:
mapping_path =  "f://" + Path(MAPPING_FILE).resolve().__str__()
interval_path = "f:/" + Path(INTERVAL_FILE).resolve().__str__()
vcf_dir_path = Path(VCF_DIR).resolve()
tmp_dir_path = Path(TMP_DIR).resolve()

In [ ]:
mapping = pd.read_csv(mapping_path, sep = "\t").set_index("HGNC", drop=False).loc[GENE,:].to_dict()

In [ ]:
vcf_files = lookup_vcfs(mapping = mapping, vcfdir = VCF_DIR, gene = "OPRM1", version = VCF_VERSION)

In [ ]:
cp_file = (tmp_dir_path / Path(f"{GENE}.cp.mt")).__str__()

mt = import_mt(vcf_files.get("vcfs"), mapping).checkpoint(cp_file)

In [ ]:
mt = downsample_mt(mt, DOWNSAMPLE_P)